This notebook is for checking if there are Gs at the editing sites positions and changing it to A's.

In [54]:
import pandas as pd
import numpy as np
import os

#a function to parse the fasta file provided by Coleoid Database authors

def parse_fasta_file(filename):
    with open(filename) as f:
        lines = f.readlines()

    data = []
    for i in range(0, len(lines), 2):
        line1 = lines[i].strip()
        line2 = lines[i+1].strip()
        if line1.startswith(">"):
            trinity_id, name1, orf_start, name2, orf_end, name3, strand, info = line1[1:].split("\t")
            sequence = line2
            data.append([trinity_id, int(orf_start), int(orf_end), strand, info, sequence])

    df = pd.DataFrame(data, columns=["Trinity id", "OrfStart", "OrfEnd", "Strand", "Info", "Sequence"])
    return df


fasta_df = parse_fasta_file('../transcriptomes/orfs_Dpaelii.fa')

In [41]:
fasta_df

,Trinity id,OrfStart,OrfEnd,Strand,Info,Sequence
0,comp111794_c0_seq1,79,1044,+,sp|A6ND36|FA83G_HUMAN Protein FAM83G OS=Homo s...,GCTTAAAAAAAGAAAGAAAAAAGAGAGAAAGATAACTCAAGTGAGA...
1,comp262601_c0_seq4,978,2528,+,sp|Q5IS76|ACHA6_PANTR Neuronal acetylcholine r...,ATCATCATCGTCGTCGTCGTCATCATCATTATCATCATCGATATCT...
2,comp267117_c0_seq25,16,519,+,sp|Q53H47|SETMR_HUMAN Histone-lysine N-methylt...,AAACAAGGAAACTGATCGTTTTTACTTTCTGGACATGAAGAAGAAA...
3,comp237873_c1_seq1,18,1424,+,sp|Q80U78|PUM1_MOUSE Pumilio homolog 1 OS=Mus ...,TCTTCGCCATGTTGTGAATTGGTTGTTATACAGCGTTATCTTTATC...
4,comp317335_c0_seq1,3,593,+,"sp|P30044|PRDX5_HUMAN Peroxiredoxin-5, mitocho...",ACGTGCCATATTCCTGTGTAAGCATGGCGTGTTTGCTGCAGTCTCT...
...,...,...,...,...,...,...
14499,comp182945_c0_seq1,77,1279,+,sp|Q92968|PEX13_HUMAN Peroxisomal membrane pro...,TAATCATAAAAAATAACAACAAAACGAAAACGAAGAAAAATAAATT...
14500,comp4827546_c0_seq1,3,212,+,sp|O69548|TOP1_MYCLE DNA topoisomerase 1 OS=My...,GCTCTTCCGATCTTTACGAGCTGGTCTGGATGCGTACCCTGGCCTC...
14501,comp235867_c1_seq1,1945,5310,+,sp|Q80U95|UBE3C_MOUSE Ubiquitin-protein ligase...,CTTTTTTCTTTTCTATTTTTTTTTTTTTTTTTCATTGCATAATTGA...
14502,comp249565_c0_seq3,1,1158,+,sp|Q8NFW1|COMA1_HUMAN Collagen alpha-1(XXII) c...,ATTAATAAAACCTGTGAGGACGTTGCTACATACGTAGCCTGCTCCG...


In [55]:
excel_file = "../editingsites/editing_sites.xlsx"
excel_df = pd.read_excel(excel_file, sheet_name="D.pea Editing sites")
excel_df.head()

excel_df['Editing location (base1)'] = excel_df['Editing location (base1)'].astype(int)

In [56]:
#from excel_df, make a smaller df with only two columns: Trinity id and Editing location (base1)

excel_df_changes = excel_df[['Trinity id', 'Editing location (base1)']]

#then, group editing location base1 by trinity id

excel_df_changes = excel_df_changes.groupby('Trinity id')['Editing location (base1)'].apply(list).reset_index(name='Editing location (base1)')


In [57]:
excel_df_changes

,Trinity id,Editing location (base1)
0,comp100191_c0_seq1,"[506, 507]"
1,comp1002049_c0_seq1,[2530]
2,comp100319_c0_seq1,"[903, 1342]"
3,comp1004149_c0_seq1,[393]
4,comp100550_c0_seq1,[126]
...,...,...
7425,comp99682_c0_seq1,"[487, 646, 678, 706, 874]"
7426,comp99796_c0_seq1,[202]
7427,comp99872_c0_seq1,"[276, 53]"
7428,comp99904_c0_seq1,"[348, 347, 345]"


In [58]:
#add a column to fasta_df with the editing location (base1) from excel_df_changes based on the trinity id

fasta_df['Editing location (base1)'] = fasta_df['Trinity id'].map(excel_df_changes.set_index('Trinity id')['Editing location (base1)'])

In [59]:
#drop the rows that have no editing location (base1)

fasta_df = fasta_df.dropna(subset=['Editing location (base1)'])

In [60]:
fasta_df.head()

,Trinity id,OrfStart,OrfEnd,Strand,Info,Sequence,Editing location (base1)
0,comp132287_c0_seq3,106,1338,+,sp|P21729|GRPR_MOUSE Gastrin-releasing peptide...,ATAATAATAATAATAATAATAAGGACAAAACCGAAGGAAGTCTGAC...,"[986, 693]"
2,comp139385_c0_seq2,375,1640,+,sp|Q9VZW5|FMAR_DROME FMRFamide receptor OS=Dro...,ATGCTACCTTAACTCAAATATATATATATATATAAGTATATATATA...,"[538, 539, 557, 575, 648, 1119, 1228, 1233, 12..."
5,comp115849_c0_seq2,3,671,+,"sp|Q8BJI4|PO6F2_MOUSE POU domain, class 6, tra...",AACAGCAGCAGCAGCAGCATCATAGTGTTGCCACGACGACAGTTTC...,"[53, 143, 433]"
9,comp141076_c2_seq1,1,4683,+,sp|Q8WUU4|ZN296_HUMAN Zinc finger protein 296 ...,ACAGGAGAGAAACCGTTTGAGTGCGTCTTCTGTCTCCGTAGATTTG...,"[2052, 2659, 2677]"
10,comp139398_c1_seq1,210,3764,+,sp|Q5VT52|RPRD2_HUMAN Regulation of nuclear pr...,GCGGATAGCACAAATGTGTGAGGTTGTAATGTACGATGAGAGAAGA...,"[1534, 2033, 2036, 3414]"


In [61]:
#make a new column called letter at the position. This column will have a list of the letters at the editing location (base1)
#That is how it is going to be calculated: for element in the list of editing location (base1), get the letter at that position -1

fasta_df['Letter at the position'] = fasta_df.apply(lambda row: [row['Sequence'][int(i)-1] for i in row['Editing location (base1)']], axis=1)

In [62]:
fasta_df

,Trinity id,OrfStart,OrfEnd,Strand,Info,Sequence,Editing location (base1),Letter at the position
0,comp132287_c0_seq3,106,1338,+,sp|P21729|GRPR_MOUSE Gastrin-releasing peptide...,ATAATAATAATAATAATAATAAGGACAAAACCGAAGGAAGTCTGAC...,"[986, 693]","[A, A]"
2,comp139385_c0_seq2,375,1640,+,sp|Q9VZW5|FMAR_DROME FMRFamide receptor OS=Dro...,ATGCTACCTTAACTCAAATATATATATATATATAAGTATATATATA...,"[538, 539, 557, 575, 648, 1119, 1228, 1233, 12...","[A, A, A, A, A, A, A, A, A, A]"
5,comp115849_c0_seq2,3,671,+,"sp|Q8BJI4|PO6F2_MOUSE POU domain, class 6, tra...",AACAGCAGCAGCAGCAGCATCATAGTGTTGCCACGACGACAGTTTC...,"[53, 143, 433]","[A, A, A]"
9,comp141076_c2_seq1,1,4683,+,sp|Q8WUU4|ZN296_HUMAN Zinc finger protein 296 ...,ACAGGAGAGAAACCGTTTGAGTGCGTCTTCTGTCTCCGTAGATTTG...,"[2052, 2659, 2677]","[A, A, A]"
10,comp139398_c1_seq1,210,3764,+,sp|Q5VT52|RPRD2_HUMAN Regulation of nuclear pr...,GCGGATAGCACAAATGTGTGAGGTTGTAATGTACGATGAGAGAAGA...,"[1534, 2033, 2036, 3414]","[A, A, A, A]"
...,...,...,...,...,...,...,...,...
11957,comp141705_c0_seq1,3,5099,+,sp|Q13219|PAPP1_HUMAN Pappalysin-1 OS=Homo sap...,ATGATGATGATGATGATGACATCTGGGCCAAAAATATCAAACCAGC...,"[481, 484, 508, 734, 746, 812, 836, 1182, 1214...","[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ..."
11958,comp83590_c0_seq1,71,2236,+,sp|Q5F415|TBC23_CHICK TBC1 domain family membe...,ATTGTTTTCAATCATGTGATGTATAAAAGTTAATCTATTTGATTTG...,"[473, 639, 812, 899, 908, 929, 971, 1099, 1397...","[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ..."
11959,comp124002_c0_seq1,493,1671,+,sp|Q5W9T5|OPR_OCTVU Octopressin receptor OS=Oc...,TACAACCTGGAATTTGGGATTTAAAAACCCAAACAAAAAAAAAACT...,[1477],[A]
11961,comp134205_c0_seq1,6,1436,+,sp|Q52I78|NAMPT_PIG Nicotinamide phosphoribosy...,ATTGACCATACAAAATGGCAGAGTGCATGGCAGGCGGGACCGATAA...,"[801, 825, 1024]","[A, A, A]"


In [63]:
#display the rows if in the list of the letter at the position there is at least one G

fasta_G = fasta_df[fasta_df['Letter at the position'].apply(lambda x: 'G' in x)]

In [64]:
fasta_G

,Trinity id,OrfStart,OrfEnd,Strand,Info,Sequence,Editing location (base1),Letter at the position


In [7]:
excel_df_changes

,Trinity id,Editing location (base1)
0,comp100000_c0_seq1,"[1425, 1426, 2613]"
1,comp100004_c0_seq1,"[163, 277, 449]"
2,comp100010_c1_seq2,"[3489, 3450, 3402, 3082, 2890, 2874, 2868, 271..."
3,comp100016_c0_seq1,[585]
4,comp100017_c0_seq1,"[635, 1244]"
...,...,...
7647,comp99972_c0_seq1,"[1583, 1566, 1563, 1555, 1552, 1515, 1510, 143..."
7648,comp99986_c0_seq2,"[840, 1223, 1379, 1466, 1467, 1569, 1576, 2447..."
7649,comp99991_c0_seq1,[1990]
7650,comp99997_c0_seq1,[139]


In [9]:
#for each Trinity id in fasta_df and for each editing location in excel_df_changes, update the fasta_df with the new sequence, where the sequence[editing location - 1] is changed to 'G'. If the sequence[editing location - 1] is not 'A', then print 'editing site {} is not an A in trinity id {}'.

for index, row in fasta_df.iterrows():
    if row['Trinity id'] in excel_df_changes['Trinity id'].values:
        for i in excel_df_changes.loc[excel_df_changes['Trinity id'] == row['Trinity id']]['Editing location (base1)'].values[0]:
            if row['Sequence'][i - 1] == 'A':
                fasta_df.loc[index, 'Sequence'] = row['Sequence'][:i - 1] + 'G' + row['Sequence'][i:]
            else:
                print('editing site {} is not an A in trinity id {}'.format(i, row['Trinity id']))

editing site 1320 is not an A in trinity id comp165306_c0_seq6
editing site 2013 is not an A in trinity id comp132291_c0_seq2
editing site 1414 is not an A in trinity id comp155190_c0_seq4
editing site 922 is not an A in trinity id comp133357_c0_seq2
editing site 503 is not an A in trinity id comp167079_c0_seq1
editing site 2421 is not an A in trinity id comp134050_c0_seq1
editing site 307 is not an A in trinity id comp164498_c0_seq6
editing site 522 is not an A in trinity id comp239534_c0_seq1
editing site 68 is not an A in trinity id comp193490_c0_seq1
editing site 726 is not an A in trinity id comp150749_c0_seq1
editing site 1790 is not an A in trinity id comp145891_c0_seq2
editing site 2231 is not an A in trinity id comp142967_c0_seq1
editing site 1842 is not an A in trinity id comp177562_c0_seq1
editing site 1386 is not an A in trinity id comp152055_c0_seq2
editing site 1613 is not an A in trinity id comp84312_c0_seq1
editing site 1355 is not an A in trinity id comp140629_c0_seq1


In [12]:
#create new updates fasta df, where there are only Trinity id present in excel_df_changes

fasta_df_updated = fasta_df[fasta_df['Trinity id'].isin(excel_df_changes['Trinity id'])]

In [14]:
fasta_df

,Trinity id,OrfStart,OrfEnd,Strand,Info,Sequence
0,comp2554101_c0_seq1,3,374,+,sp|Q802W4|NOL10_DANRE Nucleolar protein 10 OS=...,GTTCAGACCAGCCTGACATTGAGACAATATTTGTTCCCGCGTTTGG...
1,comp165306_c0_seq6,475,1479,+,sp|Q8TC12|RDH11_HUMAN Retinol dehydrogenase 11...,GTAGATGGAAACTGAAAGAAGTCCGTCATATATATATCTATGTGTG...
2,comp130941_c0_seq1,1261,1824,+,sp|Q9UGQ2|FLOWR_HUMAN Calcium channel flower h...,TTTTTGCGTGGAATCAAGTACCCAGACCTAATATAATGCAAACCCC...
3,comp132291_c0_seq2,642,2444,+,sp|Q6TXF1|ARRD3_RAT Arrestin domain-containing...,TATAAATATATATACACACACACATATAAAAAAAACATCAAGTAGT...
4,comp155190_c0_seq4,709,1425,+,sp|Q5STE3|FSTL4_MOUSE Follistatin-related prot...,TGTTGGTGTATGCATGCATGCATGTGTGCGTGTGTGTGGGTGCATG...
...,...,...,...,...,...,...
17519,comp10468_c0_seq1,2,358,+,sp|Q3MQ24|ATG5_BOVIN Autophagy protein 5 OS=Bo...,GGCTCAGTTGTGGTATTGCCTGCTTTGCGCTGACTTTGATGGCTAC...
17520,comp90405_c0_seq1,1,309,+,sp|P17038|ZNF43_HUMAN Zinc finger protein 43 O...,ATCTGTGATAAACCATTCTCTGAATATCATGTCTTCGTGAAACATA...
17521,comp152009_c0_seq1,3,2015,+,sp|Q6AXU1|BTBDG_RAT BTB/POZ domain-containing ...,TAAAAAAAAAAAAGCGAAAAAAAAAAACGATGCGGAAACAAACCAA...
17522,comp99962_c0_seq1,303,1289,+,sp|Q90W58|DUS1B_XENLA Dual specificity protein...,AGAGAGAGAGAGAGAAAGAAAGTGAGGGTGGTAGAGAGAGGAAGGG...


In [27]:
#for trinity id in fasta_df_updated, for comp165306_c0_seq6, print the sequence[816]

for index, row in fasta_df_updated.iterrows():
    if row['Trinity id'] == 'comp178061_c1_seq1':
        print(row['Sequence'][387])

G


In [55]:
df_ovul = pd.read_csv('blast_excel_df_Ovulgaris.csv')

In [60]:
#print rows where sseqid is empty

df_ovul[df_ovul['sseqid'].isnull()]

,Trinity id,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,Editing location RNA,original AA,edited AA


In [59]:
df_ovul.to_excel('blast_excel_df_Ovulgaris.xlsx', index=False)

In [ ]:
smlf = df_ovul['sseqid', ]